In [1]:
import numpy as np
import pandas as pd
import datetime
from scipy import stats
from sklearn.preprocessing import normalize, Imputer

# Importing Train Data

In [2]:
df_data_train = pd.read_csv('raw_data_70_new.csv', engine= 'python')
df_account_train = pd.read_csv('raw_account_70_new.csv', engine = 'python', parse_dates = ['dt_opened','upload_dt', 'opened_dt', 'paymt_str_dt', 'paymt_end_dt','last_paymt_dt','closed_dt','reporting_dt'])
df_enquiry_train = pd.read_csv('raw_enquiry_70_new.csv', engine = 'python', parse_dates = ['dt_opened','upload_dt', 'enquiry_dt'])

In [3]:
def findMode(arr):
    return (stats.mode(arr)).mode

In [4]:
def paymthistorylength(x):
    count=0
    for index, str in np.ndenumerate(x):
        str = str.strip('"""')
        count+=len(str)
    return count/len(x)

In [5]:
def countAccount(x):
    count=0
    for index, item in np.ndenumerate(x.values):
        if item in [1,2,3,4,5,6,7,8,9,13,15,17,32,33,34,41,41,42,51,54,59,60]:
            count+=1
    return count/len(x)

In [6]:
def frequencyOfDPD(arr):
    value = []
    count=0
    for index, x in np.ndenumerate(arr):
        dpdlist=[]
        x = x.strip('"""')
        f = [x[i:i+3] for i in range(0,len(x),3)]
        for each in f:
            try:
                dpdlist.append(int(each))
                continue
            except(ValueError, Exception) as e:
                continue
        if dpdlist:
            value.append(len(dpdlist))
        else:
            value.append(0)
    return value

In [7]:
def latestNdpd(arr,freq=None):
    value = []
    for index, x in np.ndenumerate(arr):
        dpdlist=[]
        count=0
        x = x.strip('"""')
        f = [x[i:i+3] for i in range(0,len(x),3)]
        for each in f:
            try:
                dpdlist.append(int(each))
                continue
            except(ValueError, Exception) as e:
                dpdlist.append(0)
                continue
        if dpdlist:
            if freq==None:
                for dpd in dpdlist:
                    count+=dpd
                value.append(count)
            else:
                try:
                    if dpdlist[freq-1]!=0:
                        value.append(1)
                        continue
                    else:
                        value.append(0)
                except(IndexError) as e:
                    value.append(0)
                    continue
        else:
            value.append(0)
    return value

In [8]:
def countCredit(x):
    count=0
    for index, item in np.ndenumerate(x.values):
        if item in [10,35,14,31]:
            count+=1
    return count/len(x)

In [9]:
df_data_train.isnull().sum()

dt_opened          0
customer_no        0
entry_time        15
feature_1         15
feature_2       2836
feature_3       2836
feature_4         15
feature_5         15
feature_6         15
feature_7         15
feature_8      22635
feature_9      22635
feature_10     23845
feature_11        15
feature_12        15
feature_13     13004
feature_14      7733
feature_15        23
feature_16        27
feature_17     22869
feature_18     23878
feature_19        15
feature_20         0
feature_21        15
feature_22         0
feature_23        15
feature_24       640
feature_25        15
feature_26        15
feature_27      3637
               ...  
feature_51     11422
feature_52        15
feature_53     11610
feature_54        15
feature_55        15
feature_56        15
feature_57     21503
feature_58        15
feature_59        15
feature_60        15
feature_61     23887
feature_62        15
feature_63        15
feature_64        15
feature_65        15
feature_66        15
feature_67   

In [10]:
df_enquiry_train.isnull().sum()

dt_opened        0
customer_no      0
upload_dt      110
enquiry_dt     110
enq_purpose    110
enq_amt        110
dtype: int64

In [11]:
df_account_train.isnull().sum()

dt_opened                   0
customer_no                 0
upload_dt                   0
acct_type                   0
owner_indic                 0
opened_dt                 455
last_paymt_dt           25487
closed_dt              109075
reporting_dt                0
high_credit_amt          8875
cur_balance_amt             0
amt_past_due           185453
paymenthistory1             0
paymenthistory2        107824
paymt_str_dt                1
paymt_end_dt                1
creditlimit            137477
cashlimit              151047
rateofinterest         161496
paymentfrequency       122436
actualpaymentamount    145276
dtype: int64

In [12]:
newdf_train = df_account_train
newdf1_train = df_enquiry_train

In [13]:
#  Assuming current date for null last_paymt_dt.
newdf_train.last_paymt_dt.fillna(datetime.datetime.now(),inplace=True)

In [14]:
# Assuming value of 0.0001 for null high_credit_amt and then high_credit_amt is assumed for null creditlimit.
newdf_train.high_credit_amt.fillna(0.0001, inplace=True)
newdf_train.creditlimit.fillna(newdf_train.high_credit_amt, inplace=True)

In [15]:
newdf_train['diff_opened_lastPaymt_dt'] = newdf_train['opened_dt'].sub(newdf_train['last_paymt_dt'], axis=0)
newdf_train['diff_opened_lastPaymt_dt'] = newdf_train['diff_opened_lastPaymt_dt'] / np.timedelta64(1, 'D')
newdf_train.isnull().sum(axis=0)

dt_opened                        0
customer_no                      0
upload_dt                        0
acct_type                        0
owner_indic                      0
opened_dt                      455
last_paymt_dt                    0
closed_dt                   109075
reporting_dt                     0
high_credit_amt                  0
cur_balance_amt                  0
amt_past_due                185453
paymenthistory1                  0
paymenthistory2             107824
paymt_str_dt                     1
paymt_end_dt                     1
creditlimit                      0
cashlimit                   151047
rateofinterest              161496
paymentfrequency            122436
actualpaymentamount         145276
diff_opened_lastPaymt_dt       455
dtype: int64


# Deriving Train Dataset Account Attriutes

In [16]:
newdf_train['highcr_creditlimt'] = newdf_train['high_credit_amt'] - newdf_train['creditlimit']
newdf_train['currBal_highcr'] = newdf_train['cur_balance_amt'] - newdf_train['high_credit_amt']
newdf_train['currbal_creditlimit'] = newdf_train['cur_balance_amt'] - newdf_train['creditlimit']


In [17]:
newdf_train['1DPDReported'] = latestNdpd(newdf_train['paymenthistory1'].values, freq=1)
newdf_train['2DPDReported'] = newdf_train['1DPDReported'] | latestNdpd(newdf_train['paymenthistory1'].values, freq=2)
newdf_train['3DPDReported'] = newdf_train['2DPDReported'] | latestNdpd(newdf_train['paymenthistory1'].values, freq=3)
newdf_train['totalDPD'] = latestNdpd(newdf_train['paymenthistory1'].values)
newdf_train['frequencyofDPDreported'] = frequencyOfDPD(newdf_train['paymenthistory1'].values)

In [18]:
group1_train = newdf_train.groupby('customer_no', as_index=False).agg({'acct_type': ['count'],'diff_opened_lastPaymt_dt': ['mean','sum'],'highcr_creditlimt': ['mean','sum'],'currBal_highcr': ['mean','sum'], 'currbal_creditlimit': ['mean','sum'], '1DPDReported': ['mean','sum'], '2DPDReported': ['mean','sum'], '3DPDReported': ['mean','sum'], 'totalDPD': ['mean','sum'], 'frequencyofDPDreported': ['mean','sum'], 'cur_balance_amt': ['mean','sum','std','count'], 'creditlimit': ['mean','sum','std'], 'high_credit_amt': ['mean','sum','std']})
group1_train.columns.values

array([('customer_no', ''), ('acct_type', 'count'),
       ('diff_opened_lastPaymt_dt', 'mean'),
       ('diff_opened_lastPaymt_dt', 'sum'), ('highcr_creditlimt', 'mean'),
       ('highcr_creditlimt', 'sum'), ('currBal_highcr', 'mean'),
       ('currBal_highcr', 'sum'), ('currbal_creditlimit', 'mean'),
       ('currbal_creditlimit', 'sum'), ('1DPDReported', 'mean'),
       ('1DPDReported', 'sum'), ('2DPDReported', 'mean'),
       ('2DPDReported', 'sum'), ('3DPDReported', 'mean'),
       ('3DPDReported', 'sum'), ('totalDPD', 'mean'), ('totalDPD', 'sum'),
       ('frequencyofDPDreported', 'mean'),
       ('frequencyofDPDreported', 'sum'), ('cur_balance_amt', 'mean'),
       ('cur_balance_amt', 'sum'), ('cur_balance_amt', 'std'),
       ('cur_balance_amt', 'count'), ('creditlimit', 'mean'),
       ('creditlimit', 'sum'), ('creditlimit', 'std'),
       ('high_credit_amt', 'mean'), ('high_credit_amt', 'sum'),
       ('high_credit_amt', 'std')], dtype=object)

In [19]:
df_derived_train = pd.DataFrame()
df_derived_train['customer_no'] = group1_train['customer_no']

In [20]:
df_derived_train['diff_opened_lastPaymt_dt_sum'] = group1_train[('diff_opened_lastPaymt_dt','sum')]
df_derived_train['diff_highcr_creditlim_mean'] = group1_train[('highcr_creditlimt','mean')]
df_derived_train['1DPDReported_mean'] = group1_train[('1DPDReported','mean')]
df_derived_train['2DPDReported_mean'] = group1_train[('2DPDReported','mean')]
df_derived_train['3DPDReported_mean'] = group1_train[('3DPDReported','mean')]
df_derived_train['totalDPD_sum'] = group1_train[('3DPDReported','sum')]
df_derived_train['frequencyofDPDreported_mean'] = group1_train[('frequencyofDPDreported','mean')]

df_derived_train['ratio_totalCurrbal_totalcrlim'] = group1_train[('cur_balance_amt','sum')]/group1_train[('creditlimit','sum')]
df_derived_train['ratio_totalCurrbal_totalhighCr'] = group1_train[('cur_balance_amt','sum')]/group1_train[('high_credit_amt','sum')]
df_derived_train['ratio_totalhighcr_totalcrlim'] = group1_train[('high_credit_amt','sum')]/group1_train[('creditlimit','sum')]
df_derived_train.isnull().sum().sum()

0

In [21]:
d1_train = newdf_train.groupby('customer_no',as_index=False)['acct_type'].apply(countCredit).reset_index(name='creditcount')
df_derived_train['avg_creditcount'] = d1_train.creditcount

In [22]:
d1_train = newdf_train.groupby('customer_no',as_index=False)['acct_type'].apply(countAccount).reset_index(name='loancount')
df_derived_train['avg_loancount'] = d1_train.loancount/group1_train[('acct_type','count')]

In [23]:
d1_train = newdf_train.groupby('customer_no',as_index=False)['paymenthistory1'].apply(paymthistorylength).reset_index(name='historylength')
df_derived_train['avg_payhistlength'] = d1_train.historylength

In [24]:
d_1_train = newdf_train[newdf_train['high_credit_amt'] > newdf_train['creditlimit']]
g_1_train = d_1_train.groupby('customer_no').size().reset_index(name='count')
g_2_train = newdf_train.groupby('customer_no').size().reset_index(name='count')
l_train = (g_2_train[~g_2_train['customer_no'].isin(g_1_train['customer_no'])==True].customer_no).tolist()
d_train = pd.DataFrame()
d_train['customer_no'] = l_train
d_train['count'] = [0 for i in range(len(d_train))]
d1_train = pd.concat([g_1_train, d_train], ignore_index=True)
d1_train.sort_values('customer_no',inplace=True)
d1_train.reset_index(drop=True,inplace=True)
d1_train['meancount'] = d1_train['count']/group1_train[('acct_type','count')]
df_derived_train['meanAcctWithHighCrGreaterThanCreditLim'] = d1_train['meancount']

In [25]:
d_1_train = newdf_train[newdf_train['cur_balance_amt'] == 0]
g_1_train = d_1_train.groupby('customer_no').size().reset_index(name='count')
g_2_train = newdf_train.groupby('customer_no').size().reset_index(name='count')
l_train = (g_2_train[~g_2_train['customer_no'].isin(g_1_train['customer_no'])==True].customer_no).tolist()
d_train = pd.DataFrame()
d_train['customer_no'] = l_train
d_train['count'] = [0 for i in range(len(d_train))]
d1_train = pd.concat([g_1_train, d_train], ignore_index=True)
d1_train.sort_values('customer_no',inplace=True)
d1_train.reset_index(drop=True,inplace=True)
d1_train['meancount'] = d1_train['count']/group1_train[('acct_type','count')]
df_derived_train['meanAcctWithCurrBalEqualsZero'] = d1_train['count']

# Deriving Train Enquiry Attributes 

In [26]:
d1_train = newdf1_train.enquiry_dt.values
d2_train = d1_train[1:]
d2_train = np.append(d2_train,d1_train[-1])
d_train = (d2_train-d1_train)/np.timedelta64(1, 'D')
newdf1_train['GapEnquiryDates'] = abs(d_train)

In [27]:
group2_train = newdf1_train.groupby('customer_no', as_index=False).agg({'dt_opened': ['count'],'upload_dt': ['count'],'enquiry_dt': ['count'],'enq_purpose': ['count'], 'enq_amt': ['count'],'enq_purpose': findMode,'GapEnquiryDates': ['mean','sum']})

In [28]:
df_derived_train['meanGapEnquiryDates'] = group2_train[('GapEnquiryDates','mean')]

In [29]:
df_derived_train['mostFrequentEnquiryPorpose'] = group2_train[('enq_purpose','findMode')]

In [30]:
values = []
for i in group2_train.customer_no:
    values.append(newdf1_train[newdf1_train.customer_no==i].enq_purpose.values[0])
df_derived_train['mostRecentEnquiryPorpose'] = values

In [31]:
newdf2_train = df_data_train.select_dtypes(['float64', 'int64', 'float32','int32'])
for col in newdf2_train.columns.values.tolist():
    df_derived_train[col] = newdf2_train[col]

In [32]:
# Since most of the Imp Features has been derived already, Remaining features are less significant and those columns with higher Null Vlaues can be removed/drop.  

df_derived_trian = df_derived_train.drop(['feature_3','feature_74','feature_49','feature_14'], axis = 1, inplace = True)

In [33]:
df_derived_train.isnull().sum()

customer_no                                 0
diff_opened_lastPaymt_dt_sum                0
diff_highcr_creditlim_mean                  0
1DPDReported_mean                           0
2DPDReported_mean                           0
3DPDReported_mean                           0
totalDPD_sum                                0
frequencyofDPDreported_mean                 0
ratio_totalCurrbal_totalcrlim               0
ratio_totalCurrbal_totalhighCr              0
ratio_totalhighcr_totalcrlim                0
avg_creditcount                             0
avg_loancount                               0
avg_payhistlength                           0
meanAcctWithHighCrGreaterThanCreditLim      0
meanAcctWithCurrBalEqualsZero               0
meanGapEnquiryDates                       110
mostFrequentEnquiryPorpose                  0
mostRecentEnquiryPorpose                  109
feature_4                                  15
feature_6                                  15
feature_7                         

In [34]:
Final_Train = df_derived_train

In [35]:
len(df_derived_train.columns.values)

48

In [36]:
# df_derived_train.replace(np.nan,NaN,inplace=True)
df_derived_train.replace(np.inf,np.nan,inplace=True)
df_derived_train.replace(-np.inf,np.nan,inplace=True)
df_derived_train.to_csv('C:\\Users\\Admin\\Desktop\\Tookitaki\\Final_train')

#  Importing Test Data

In [37]:
df_data_test = pd.read_csv('raw_data_30_new.csv', engine= 'python')
df_account_test = pd.read_csv('raw_account_30_new.csv', engine = 'python', parse_dates = ['dt_opened','upload_dt', 'opened_dt', 'paymt_str_dt', 'paymt_end_dt','last_paymt_dt','closed_dt','reporting_dt'])
df_enquiry_test = pd.read_csv('raw_enquiry_30_new.csv', engine = 'python', parse_dates = ['dt_opened','upload_dt', 'enquiry_dt'])

In [38]:
# Since Most of the attributes in Train & Test Datasets are same, we can repeat the same pre-processing steps that has done for Train Dataset

df_data_test.isnull().sum()

dt_opened          0
customer_no        0
entry_time         9
feature_1          9
feature_2       1211
feature_3       1211
feature_4          9
feature_5          9
feature_6          9
feature_7          9
feature_8       9708
feature_9       9708
feature_10     10220
feature_11         9
feature_12         9
feature_13      5518
feature_14      3263
feature_15        14
feature_16        12
feature_17      9812
feature_18     10227
feature_19         9
feature_20         0
feature_21         9
feature_22         0
feature_23         9
feature_24       276
feature_25         9
feature_26         9
feature_27      1533
               ...  
feature_51      4916
feature_52         9
feature_53      4984
feature_54         9
feature_55         9
feature_56         9
feature_57      9202
feature_58         9
feature_59         9
feature_60         9
feature_61     10234
feature_62         9
feature_63         9
feature_64         9
feature_65         9
feature_66         9
feature_67   

In [40]:
df_enquiry_test.isnull().sum()

dt_opened       0
customer_no     0
upload_dt      42
enquiry_dt     42
enq_purpose    42
enq_amt        42
dtype: int64

In [41]:
df_account_test.isnull().sum()

dt_opened                  0
customer_no                0
upload_dt                  0
acct_type                  0
owner_indic                0
opened_dt                174
last_paymt_dt          10703
closed_dt              46643
reporting_dt               0
high_credit_amt         3695
cur_balance_amt            0
amt_past_due           79170
paymenthistory1            0
paymenthistory2        46299
paymt_str_dt               1
paymt_end_dt               1
creditlimit            59076
cashlimit              64823
rateofinterest         69061
paymentfrequency       52537
actualpaymentamount    62305
dtype: int64

In [42]:
newdf_test = df_account_test
newdf1_test = df_enquiry_test

In [43]:
#  Assuming current date for null last_paymt_dt.
newdf_test.last_paymt_dt.fillna(datetime.datetime.now(),inplace=True)

In [44]:
# Assuming value of 0.0001 for null high_credit_amt and then high_credit_amt is assumed for null creditlimit.
newdf_test.high_credit_amt.fillna(0.0001, inplace=True)
newdf_test.creditlimit.fillna(newdf_test.high_credit_amt, inplace=True)

In [45]:
newdf_test['diff_opened_lastPaymt_dt'] = newdf_test['opened_dt'].sub(newdf_test['last_paymt_dt'], axis=0)
newdf_test['diff_opened_lastPaymt_dt'] = newdf_test['diff_opened_lastPaymt_dt'] / np.timedelta64(1, 'D')
newdf_test.isnull().sum(axis=0)

dt_opened                       0
customer_no                     0
upload_dt                       0
acct_type                       0
owner_indic                     0
opened_dt                     174
last_paymt_dt                   0
closed_dt                   46643
reporting_dt                    0
high_credit_amt                 0
cur_balance_amt                 0
amt_past_due                79170
paymenthistory1                 0
paymenthistory2             46299
paymt_str_dt                    1
paymt_end_dt                    1
creditlimit                     0
cashlimit                   64823
rateofinterest              69061
paymentfrequency            52537
actualpaymentamount         62305
diff_opened_lastPaymt_dt      174
dtype: int64

# Deriving Test Dataset Account Attriutes

In [46]:
newdf_test['highcr_creditlimt'] = newdf_test['high_credit_amt'] - newdf_test['creditlimit']
newdf_test['currBal_highcr'] = newdf_test['cur_balance_amt'] - newdf_test['high_credit_amt']
newdf_test['currbal_creditlimit'] = newdf_test['cur_balance_amt'] - newdf_test['creditlimit']


In [47]:
newdf_test['1DPDReported'] = latestNdpd(newdf_test['paymenthistory1'].values, freq=1)
newdf_test['2DPDReported'] = newdf_test['1DPDReported'] | latestNdpd(newdf_test['paymenthistory1'].values, freq=2)
newdf_test['3DPDReported'] = newdf_test['2DPDReported'] | latestNdpd(newdf_test['paymenthistory1'].values, freq=3)
newdf_test['totalDPD'] = latestNdpd(newdf_test['paymenthistory1'].values)
newdf_test['frequencyofDPDreported'] = frequencyOfDPD(newdf_test['paymenthistory1'].values)

In [48]:
group1_test = newdf_test.groupby('customer_no', as_index=False).agg({'acct_type': ['count'],'diff_opened_lastPaymt_dt': ['mean','sum'],'highcr_creditlimt': ['mean','sum'],'currBal_highcr': ['mean','sum'], 'currbal_creditlimit': ['mean','sum'], '1DPDReported': ['mean','sum'], '2DPDReported': ['mean','sum'], '3DPDReported': ['mean','sum'], 'totalDPD': ['mean','sum'], 'frequencyofDPDreported': ['mean','sum'], 'cur_balance_amt': ['mean','sum','std','count'], 'creditlimit': ['mean','sum','std'], 'high_credit_amt': ['mean','sum','std']})
group1_test.columns.values

array([('customer_no', ''), ('acct_type', 'count'),
       ('diff_opened_lastPaymt_dt', 'mean'),
       ('diff_opened_lastPaymt_dt', 'sum'), ('highcr_creditlimt', 'mean'),
       ('highcr_creditlimt', 'sum'), ('currBal_highcr', 'mean'),
       ('currBal_highcr', 'sum'), ('currbal_creditlimit', 'mean'),
       ('currbal_creditlimit', 'sum'), ('1DPDReported', 'mean'),
       ('1DPDReported', 'sum'), ('2DPDReported', 'mean'),
       ('2DPDReported', 'sum'), ('3DPDReported', 'mean'),
       ('3DPDReported', 'sum'), ('totalDPD', 'mean'), ('totalDPD', 'sum'),
       ('frequencyofDPDreported', 'mean'),
       ('frequencyofDPDreported', 'sum'), ('cur_balance_amt', 'mean'),
       ('cur_balance_amt', 'sum'), ('cur_balance_amt', 'std'),
       ('cur_balance_amt', 'count'), ('creditlimit', 'mean'),
       ('creditlimit', 'sum'), ('creditlimit', 'std'),
       ('high_credit_amt', 'mean'), ('high_credit_amt', 'sum'),
       ('high_credit_amt', 'std')], dtype=object)

In [49]:
df_derived_test = pd.DataFrame()
df_derived_test['customer_no'] = group1_test['customer_no']

In [50]:
df_derived_test['diff_opened_lastPaymt_dt_sum'] = group1_test[('diff_opened_lastPaymt_dt','sum')]
df_derived_test['diff_highcr_creditlim_mean'] = group1_test[('highcr_creditlimt','mean')]
df_derived_test['1DPDReported_mean'] = group1_test[('1DPDReported','mean')]
df_derived_test['2DPDReported_mean'] = group1_test[('2DPDReported','mean')]
df_derived_test['3DPDReported_mean'] = group1_test[('3DPDReported','mean')]
df_derived_test['totalDPD_sum'] = group1_test[('3DPDReported','sum')]
df_derived_test['frequencyofDPDreported_mean'] = group1_test[('frequencyofDPDreported','mean')]

df_derived_test['ratio_totalCurrbal_totalcrlim'] = group1_test[('cur_balance_amt','sum')]/group1_test[('creditlimit','sum')]
df_derived_test['ratio_totalCurrbal_totalhighCr'] = group1_test[('cur_balance_amt','sum')]/group1_test[('high_credit_amt','sum')]
df_derived_test['ratio_totalhighcr_totalcrlim'] = group1_test[('high_credit_amt','sum')]/group1_test[('creditlimit','sum')]
df_derived_test.isnull().sum().sum()

0

In [51]:
d1_test = newdf_test.groupby('customer_no',as_index=False)['acct_type'].apply(countCredit).reset_index(name='creditcount')
df_derived_test['avg_creditcount'] = d1_test.creditcount

In [52]:
d1_test = newdf_test.groupby('customer_no',as_index=False)['acct_type'].apply(countAccount).reset_index(name='loancount')
df_derived_test['avg_loancount'] = d1_test.loancount/group1_test[('acct_type','count')]

In [53]:
d1_test = newdf_test.groupby('customer_no',as_index=False)['paymenthistory1'].apply(paymthistorylength).reset_index(name='historylength')
df_derived_test['avg_payhistlength'] = d1_test.historylength

In [54]:
d_1_test = newdf_test[newdf_test['high_credit_amt'] > newdf_test['creditlimit']]
g_1_test = d_1_test.groupby('customer_no').size().reset_index(name='count')
g_2_test = newdf_test.groupby('customer_no').size().reset_index(name='count')
l_test = (g_2_test[~g_2_test['customer_no'].isin(g_1_test['customer_no'])==True].customer_no).tolist()
d_test = pd.DataFrame()
d_test['customer_no'] = l_test
d_test['count'] = [0 for i in range(len(d_test))]
d1_test = pd.concat([g_1_test, d_test], ignore_index=True)
d1_test.sort_values('customer_no',inplace=True)
d1_test.reset_index(drop=True,inplace=True)
d1_test['meancount'] = d1_test['count']/group1_test[('acct_type','count')]
df_derived_test['meanAcctWithHighCrGreaterThanCreditLim'] = d1_test['meancount']

In [55]:
d_1_test = newdf_test[newdf_test['cur_balance_amt'] == 0]
g_1_test = d_1_test.groupby('customer_no').size().reset_index(name='count')
g_2_test = newdf_test.groupby('customer_no').size().reset_index(name='count')
l_test = (g_2_test[~g_2_test['customer_no'].isin(g_1_test['customer_no'])==True].customer_no).tolist()
d_test = pd.DataFrame()
d_test['customer_no'] = l_test
d_test['count'] = [0 for i in range(len(d_test))]
d1_test = pd.concat([g_1_test, d_test], ignore_index=True)
d1_test.sort_values('customer_no',inplace=True)
d1_test.reset_index(drop=True,inplace=True)
d1_test['meancount'] = d1_test['count']/group1_test[('acct_type','count')]
df_derived_test['meanAcctWithCurrBalEqualsZero'] = d1_test['count']

# Deriving Test Enquiry Attributes 

In [56]:
d1_test = newdf1_test.enquiry_dt.values
d2_test = d1_test[1:]
d2_test = np.append(d2_test,d1_test[-1])
d_test = (d2_test-d1_test)/np.timedelta64(1, 'D')
newdf1_test['GapEnquiryDates'] = abs(d_test)

In [57]:
group2_test = newdf1_test.groupby('customer_no', as_index=False).agg({'dt_opened': ['count'],'upload_dt': ['count'],'enquiry_dt': ['count'],'enq_purpose': ['count'], 'enq_amt': ['count'],'enq_purpose': findMode,'GapEnquiryDates': ['mean','sum']})

In [58]:
df_derived_test['meanGapEnquiryDates'] = group2_test[('GapEnquiryDates','mean')]

In [59]:
df_derived_test['mostFrequentEnquiryPorpose'] = group2_test[('enq_purpose','findMode')]

In [60]:
values = []
for i in group2_test.customer_no:
    values.append(newdf1_test[newdf1_test.customer_no==i].enq_purpose.values[0])
df_derived_test['mostRecentEnquiryPorpose'] = values

In [61]:
newdf2_test = df_data_test.select_dtypes(['float64', 'int64', 'float32','int32'])
for col in newdf2_test.columns.values.tolist():
    df_derived_test[col] = newdf2_test[col]

In [62]:
# Since most of the Imp Features has been derived already, Remaining features are less significant and those columns with higher Null Vlaues can be removed/drop.  

df_derived_trian = df_derived_test.drop(['feature_3','feature_74','feature_49','feature_14'], axis = 1, inplace = True)

In [63]:
df_derived_test.isnull().sum()

customer_no                                0
diff_opened_lastPaymt_dt_sum               0
diff_highcr_creditlim_mean                 0
1DPDReported_mean                          0
2DPDReported_mean                          0
3DPDReported_mean                          0
totalDPD_sum                               0
frequencyofDPDreported_mean                0
ratio_totalCurrbal_totalcrlim              0
ratio_totalCurrbal_totalhighCr             0
ratio_totalhighcr_totalcrlim               0
avg_creditcount                            0
avg_loancount                              0
avg_payhistlength                          0
meanAcctWithHighCrGreaterThanCreditLim     0
meanAcctWithCurrBalEqualsZero              0
meanGapEnquiryDates                       43
mostFrequentEnquiryPorpose                 0
mostRecentEnquiryPorpose                  42
feature_4                                  9
feature_6                                  9
feature_7                                  9
feature_19

In [64]:
Final_Test = df_derived_test

In [65]:
len(df_derived_test.columns.values)

48

In [66]:
# df_derived_test.replace(np.nan,NaN,inplace=True)
df_derived_test.replace(np.inf,np.nan,inplace=True)
df_derived_test.replace(-np.inf,np.nan,inplace=True)
df_derived_test.to_csv('C:\\Users\\Admin\\Desktop\\Tookitaki\\Final_test')

In [67]:
import csv
from csv import DictReader
import seaborn
import imblearn

In [68]:
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, export_graphviz, export 
from sklearn.grid_search import GridSearchCV

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [69]:
from sklearn.ensemble import RandomForestClassifier

In [70]:
imp1 = Imputer(missing_values="NaN", strategy='mean', axis=0)
imp2 = Imputer(missing_values="NaN", strategy='mean', axis=0)

In [71]:
dict_of_features = {0:'diff_opened_lastPaymt_dt_sum', 1:'diff_highcr_creditlim_mean', 2:'1DPDReported_mean', 3:'2DPDReported_mean', 4:'3DPDReported_mean', 5:'totalDPD_sum', 6:'frequencyofDPDreported_mean', 7:'ratio_totalCurrbal_totalcrlim', 8:'ratio_totalCurrbal_totalhighCr', 9:'ratio_totalhighcr_totalcrlim', 10:'avg_creditcount', 11:'avg_loancount', 12:'avg_payhistlength', 13:'meanAcctWithHighCrGreaterThanCreditLim', 14:'meanAcctWithCurrBalEqualsZero', 15:'meanGapEnquiryDates', 16:'mostFrequentEnquiryPorpose', 17:'mostRecentEnquiryPorpose', 18:'feature_3', 19:'feature_4', 20:'feature_6', 21:'feature_7', 22:'feature_14', 23:'feature_19', 24:'feature_25', 25:'feature_26', 26:'feature_29', 27:'feature_30', 28:'feature_31', 29:'feature_34', 30:'feature_35', 31:'feature_39', 32:'feature_40', 33:'feature_41', 34:'feature_42', 35:'feature_44', 36:'feature_49', 37:'feature_52', 38:'feature_55', 39:'feature_56', 40:'feature_64', 41:'feature_65', 42:'feature_66', 43:'feature_67', 44:'feature_68', 45:'feature_69', 46:'feature_71', 47:'feature_74', 48:'feature_76', 49:'feature_78'}

In [72]:
from collections import Counter
import csv
from csv import DictReader
import imblearn
from sklearn.metrics import classification_report
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn import feature_selection
from sklearn.preprocessing import normalize, Imputer
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.datasets import make_classification
import pickle
from imblearn.ensemble import EasyEnsemble
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

In [74]:
X1 = pd.read_csv('C:\\Users\\Admin\\Desktop\\Tookitaki\\Final_train.csv', engine='python')
Y_train = X1['Bad_label'].values
X1.drop(['customer_no','Bad_label'],axis=1,inplace=True)
X_train = X1.values

In [75]:
X2= pd.read_csv('C:\\Users\\Admin\\Desktop\\Tookitaki\\Final_test.csv', engine='python')
Y_test = X2['Bad_label'].values
X2.drop(['customer_no','Bad_label'],axis=1,inplace=True)
X_test = X2.values

In [76]:
imp1.fit(X_train)
X_train = imp1.transform(X_train).astype(float)
#print(X_train)
imp2.fit(X_test)
X_test = imp2.transform(X_test).astype(float)

In [77]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [78]:
print(X_train.shape)

(23896, 46)


In [79]:
bbc = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100)\
                                ,ratio='auto',replacement=False,random_state=0, bootstrap_features=False)

In [80]:
clf = SelectKBest(mutual_info_classif, k=46)
X_train = clf.fit_transform(X_train, Y_train)
X_test = clf.transform(X_test)
bbc.fit(X_train, Y_train)
y_pred = bbc.predict(X_test)

In [81]:
print(confusion_matrix(Y_test, y_pred))

[[5923 3855]
 [ 175  287]]


In [82]:
print(classification_report(Y_test,y_pred))

             precision    recall  f1-score   support

          0       0.97      0.61      0.75      9778
          1       0.07      0.62      0.12       462

avg / total       0.93      0.61      0.72     10240



In [83]:
fpr, tpr, thresholds = metrics.roc_curve(Y_test, y_pred, pos_label=1)
auc_score = metrics.auc(fpr, tpr)

In [84]:
print('auc score =',auc_score)

auc score = 0.613479858929


In [85]:
print('gini score =',2*auc_score-1)

gini score = 0.226959717858


In [86]:
X1 = pd.read_csv('C:\\Users\\Admin\\Desktop\\Tookitaki\\Final_train.csv', engine='python')
Y_train = X1['Bad_label'].values
X1.drop(['customer_no','Bad_label'],axis=1,inplace=True)
X_train = X1.values

In [87]:
X2 = pd.read_csv('C:\\Users\\Admin\\Desktop\\Tookitaki\\Final_test.csv', engine='python')
Y_test = X2['Bad_label'].values
X2.drop(['customer_no','Bad_label'],axis=1,inplace=True)
X_test = X2.values

In [88]:
imp1.fit(X_train)
X_train = imp1.transform(X_train).astype(float)

In [89]:
imp2.fit(X_test)
X_test = imp2.transform(X_test).astype(float)

In [90]:
feature_importance = {0: 0.03421852,  1: 0.00715693,  2: 0.05181835,  3: 0.01027565,  4: 0.00838411,  5: 0.0059313,  6: 0.00223466,  7: 0.00310911,  8: 0.00964993,  9: 0.01483778,  10: 0.0066048,  11: 0.00522294,  12: 0.02071259,  13: 0.00220598,  14: 0.03072879,  15: 0.00625178,  16: 0.01470893,  17: 0.02050969,  18: 0.00235906,  19: 0.00072533,  20: 0.00457309,  21: 0.01256962,  22: 0.00559733,  23: 0.0392486,  24: 0.0060228,  25: 0.00908848,  26: 0.01150884,  27: 0.01115191,  28: 0.01161009,  29: 0.01280016,  30: 0.01341943,  31: 0.01367287,  32: 0.01357383,  33: 0.01372479,  34: 0.01486814,  35: 0.01548993,  36: 0.01722397,  37: 0.01798768,  38: 0.00627185,  39: 0.00506561,  40: 0.00597463,  41: 0.00543547,  42: 0.00499733,  43: 0.00452328,  44: 0.00458949,  45: 0.0042845,  46: 0.00425256,  47: 0.00428415,  48: 0.00432716,  49: 0.00439585}#,  50: 0.00450958,  51: 0.00575839,  52: 0.00739246,  53: 0.00502165,  54: 0.0095109,  55: 0.0069166,  56: 0.0071989,  57: 0.00776389,  58: 0.00814988,  59: 0.0067113,  60: 0.00803184,  61: 0.00822991,  62: 0.00502434,  63: 0.00571724,  64: 0.00985963,  65: 0.00549805,  66: 0.00979168,  67: 0.01288979,  68: 0.01062375,  69: 0.00566553,  70: 0.00503578,  71: 0.01169688,  72: 0.00579325,  73: 0.01161525,  74: 0.01194471,  75: 0.00419361,  76: 0.00416345,  77: 0.0049956,  78: 0.01476468,  79: 0.01368045,  80: 0.00522837,  81: 0.01272689,  82: 0.00422461,  83: 0.01271347,  84: 0.00527942,  85: 0.01348907,  86: 0.01594457,  87: 0.01447534,  88: 0.00454526,  89: 0.00494497,  90: 0.01684405,  91: 0.00459594,  92: 0.00517932,  93: 0.00542252,  94: 0.01753486,  95: 0.00552958,  96: 0.00558431,  97: 0.01830974,  98: 0.01795993,  99: 0.01113913}

sorted_feature_importance = [(k, feature_importance[k]) for k in sorted(feature_importance, key=feature_importance.get, reverse=True)]

In [91]:
print(type(sorted_feature_importance))
for each in sorted_feature_importance:
        print(each)

<class 'list'>
(2, 0.05181835)
(23, 0.0392486)
(0, 0.03421852)
(14, 0.03072879)
(12, 0.02071259)
(17, 0.02050969)
(37, 0.01798768)
(36, 0.01722397)
(35, 0.01548993)
(34, 0.01486814)
(9, 0.01483778)
(16, 0.01470893)
(33, 0.01372479)
(31, 0.01367287)
(32, 0.01357383)
(30, 0.01341943)
(29, 0.01280016)
(21, 0.01256962)
(28, 0.01161009)
(26, 0.01150884)
(27, 0.01115191)
(3, 0.01027565)
(8, 0.00964993)
(25, 0.00908848)
(4, 0.00838411)
(1, 0.00715693)
(10, 0.0066048)
(38, 0.00627185)
(15, 0.00625178)
(24, 0.0060228)
(40, 0.00597463)
(5, 0.0059313)
(22, 0.00559733)
(41, 0.00543547)
(11, 0.00522294)
(39, 0.00506561)
(42, 0.00499733)
(44, 0.00458949)
(20, 0.00457309)
(43, 0.00452328)
(49, 0.00439585)
(48, 0.00432716)
(45, 0.0042845)
(47, 0.00428415)
(46, 0.00425256)
(7, 0.00310911)
(18, 0.00235906)
(6, 0.00223466)
(13, 0.00220598)
(19, 0.00072533)


In [92]:
from sklearn.ensemble import RandomForestClassifier

In [93]:
radm_clf = RandomForestClassifier(oob_score=True,n_estimators=100 )
radm_clf.fit( X_train, Y_train )

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [96]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [97]:
print(X_train.shape)
bbc = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100)\
                                ,ratio='auto',replacement=False,random_state=0, bootstrap_features=False)

(23896, 46)


In [ ]:
for i in range(11):
        temp_feat = []
        for j in range(10):
            temp_feat.append(sorted_feature_importance[(i*4)+j][0])
            X_train_new = X_train[:,temp_feat]
            X_test_new =  X_test[:,temp_feat]
            bbc.fit(X_train_new, Y_train)
            y_pred = bbc.predict(X_test_new)
            fpr, tpr, thresholds = metrics.roc_curve(Y_test, y_pred, pos_label=1)
            auc_score = metrics.auc(fpr, tpr)
        print(2*auc_score-1)